In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the training set
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values


In [3]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [4]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [5]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [7]:
# Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [8]:
# Initialising the RNN
regressor = Sequential()

In [9]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

In [10]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [11]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [12]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [13]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [14]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [15]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 10s 97ms/step - loss: 0.0449
Epoch 2/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0068
Epoch 3/100
38/38 [==============================] - 3s 84ms/step - loss: 0.0059
Epoch 4/100
38/38 [==============================] - 3s 86ms/step - loss: 0.0056
Epoch 5/100
38/38 [==============================] - 4s 93ms/step - loss: 0.0053
Epoch 6/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0050
Epoch 7/100
38/38 [==============================] - 3s 87ms/step - loss: 0.0046
Epoch 8/100
38/38 [==============================] - 3s 67ms/step - loss: 0.0052
Epoch 9/100
38/38 [==============================] - 3s 67ms/step - loss: 0.0044
Epoch 10/100
38/38 [==============================] - 3s 67ms/step - loss: 0.0043
Epoch 11/100
38/38 [==============================] - 3s 69ms/step - loss: 0.0041
Epoch 12/100
38/38 [==============================] - 3s 68ms/step - loss: 0.0046
Epoch 13/100
38/38 [===